In [1]:
import kagglehub

path = kagglehub.dataset_download("haakakak/msabyzip")

print("Path to dataset files:", path)

100%|██████████| 1.39M/1.39M [00:00<00:00, 32.0MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/haakakak/msabyzip/versions/1


In [ ]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/haakakak/msabyzip/versions/1"
print(os.listdir(dataset_path))

In [ ]:
import pandas as pd

cols = ["ZIP", "STATE", "MSA No.", "Work GPCI", "Practice Expense GPCI", "Malpractice Expense GPCI", "County No.", "MSA Name"]

file_path = "/root/.cache/kagglehub/datasets/haakakak/msabyzip/versions/1/fs11_gpci_by_msa-ZIP.xls"
dfo = pd.read_excel(file_path, names=cols, skiprows=10,index_col=False)

In [5]:
df = dfo.drop(columns=cols[1:len(cols)-1])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43508 entries, 0 to 43507
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ZIP       43508 non-null  int64  
 1   MSA Name  43497 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 679.9 KB


In [ ]:
df.dropna(inplace=True)

In [ ]:
len(df["MSA Name"].unique())

427

In [ ]:
print(df[df["MSA Name"]=="New York-Northern New Jersey-Long Island, NY-NJ-PA MSA"]["ZIP"].tolist())

[501, 544, 6390, 7001, 7002, 7003, 7004, 7005, 7006, 7007, 7008, 7009, 7010, 7011, 7012, 7013, 7014, 7015, 7016, 7017, 7018, 7019, 7020, 7021, 7022, 7023, 7024, 7026, 7027, 7028, 7029, 7030, 7031, 7032, 7033, 7034, 7035, 7036, 7039, 7040, 7041, 7042, 7043, 7044, 7045, 7046, 7047, 7050, 7051, 7052, 7054, 7055, 7057, 7058, 7059, 7060, 7061, 7062, 7063, 7064, 7065, 7066, 7067, 7068, 7069, 7070, 7071, 7072, 7073, 7074, 7075, 7076, 7077, 7078, 7079, 7080, 7081, 7082, 7083, 7086, 7087, 7088, 7090, 7091, 7092, 7093, 7094, 7095, 7096, 7097, 7099, 7101, 7102, 7103, 7104, 7105, 7106, 7107, 7108, 7109, 7110, 7111, 7112, 7114, 7175, 7182, 7184, 7188, 7189, 7191, 7192, 7193, 7194, 7195, 7197, 7198, 7199, 7201, 7202, 7203, 7204, 7205, 7206, 7207, 7208, 7302, 7303, 7304, 7305, 7306, 7307, 7308, 7309, 7310, 7311, 7390, 7395, 7399, 7401, 7403, 7405, 7407, 7410, 7416, 7417, 7418, 7419, 7420, 7421, 7422, 7423, 7424, 7428, 7430, 7432, 7435, 7436, 7438, 7439, 7440, 7442, 7444, 7446, 7450, 7451, 7452, 7456,

In [ ]:
df["MSA Name"] = df["MSA Name"].str.lower()

In [ ]:
df.head()

,ZIP,MSA Name
0,210,"boston-cambridge-quincy, ma-nh msa"
1,211,"boston-cambridge-quincy, ma-nh msa"
2,212,"boston-cambridge-quincy, ma-nh msa"
3,213,"boston-cambridge-quincy, ma-nh msa"
4,214,"boston-cambridge-quincy, ma-nh msa"


In [7]:
import requests

In [ ]:
import requests
import pandas as pd
MSA = input().strip().lower()


zip_codes = df.loc[df["MSA Name"].str.lower() == MSA, "ZIP"].tolist()

session = requests.Session()

#to get the zip code from the MSA
base_url = "https://npiregistry.cms.hhs.gov/api/?version=2.1&postal_code={}&enumeration_type=NPI-1&pretty=on"

physicians = []

for zip_code in zip_codes:
    try:
        res = session.get(base_url.format(zip_code))
        res.raise_for_status()
        data = res.json()

        for psy in data.get("results", []):
            basic_info = psy.get("basic", {})

            name = " ".join(filter(None, [
                basic_info.get("name_prefix"),
                basic_info.get("first_name"),
                basic_info.get("middle_name"),
                basic_info.get("last_name"),
                basic_info.get("credential")
            ])).strip()

            if name:
                physicians.append(name)

    except requests.RequestException as e:
        print(f"Request failed for ZIP {zip_code}: {e}")

physicians = list(set(physicians))

print(physicians)


Boston-Cambridge-Quincy, MA-NH MSA
['Dr. ANCA DANIELA PAUNICA M.D.', 'Dr. MARJORIE BRITT KOSSOFF BS, MD', 'Dr. YONG KIM DDS', '-- RUTH ADELABI MSW,CAAC', 'Dr. ARNOLD KANG M.D.', 'Mr. MARLON DE LUNA NAVARRO', 'Dr. KEYVAN KARIMIGALOUGAHI', 'Dr. KIRK MICHAEL BROWN M.D.', 'Dr. JAMES ALEXANDER PAGET DRUMMOND MBBS', 'Dr. HEIDI BRUMMERT LENNINGS Phd/MClinPsych', 'DENNIS R KERR M.D.', 'Dr. ALLISON JOYCE DUCHOW MD']


In [9]:
print(len(physicians))

12


In [ ]:
from google import genai

API_KEY = ""

def gel_locaton(prompt):
  client = genai.Client(api_key=API_KEY)
  response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
  )
  #can use pro for faster and more number of results 
  return response.text

In [ ]:
import json

details = []

for ph in physicians:
  prompt = f"""
You are an expert in healthcare navigation. Given the physician {ph['name']} located at {ph['address']}, find the associated physician group (PG).
Provide a JSON response with:
- "physician": The name of the physician.
- "latitude": The latitude of the PG.
- "longitude": The longitude of the PG.
- "group_name": The name of the physician group.
- "location_name": The location of the PG.
"""
  res = gel_locaton(prompt)
  json_start = res.find("[")
  json_end = res.rfind("]") + 1
  if json_start != -1 and json_end != -1:
    json_data = res[json_start:json_end]
    try:
        json_obj = json.loads(json_data)
        for obj in json_obj:
          details.append(obj)

    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)

else:
    print("No JSON data found in the response.")

No JSON data found in the response.


In [12]:
print(details)

[{'physician': 'ANCA DANIELA PAUNICA M.D.', 'latitude': 0.0, 'longitude': 0.0, 'group_name/name': 'Physician Group Name', 'location_name': 'Specific Location Name (e.g., Main Street Clinic)'}, {'physician': 'MARJORIE BRITT KOSSOFF', 'latitude': 42.3364, 'longitude': -71.1017, 'group_name/name': "Brigham and Women's Hospital Main Campus", 'location_name': '75 Francis Street, Boston, MA 02115'}, {'physician': 'MARJORIE BRITT KOSSOFF', 'latitude': 42.2842, 'longitude': -71.2444, 'group_name/name': "Brigham and Women's Faulkner Hospital", 'location_name': '1153 Centre Street, Boston, MA 02130'}, {'physician': 'Yong Kim DDS', 'latitude': 34.0522, 'longitude': -118.2437, 'group_name/name': 'Smile Dental Group', 'location_name': 'Downtown Los Angeles Office'}, {'physician': 'Yong Kim DDS', 'latitude': 34.1484, 'longitude': -118.0985, 'group_name/name': 'Smile Dental Group', 'location_name': 'Pasadena Office'}, {'physician': 'Ruth Adelabi', 'latitude': 34.0522, 'longitude': -118.2437, 'group_n

In [14]:
print(len(details))

20


MAPPING SERIES

In [ ]:
#WE USE FOLIUM TO MAP THE LOCATIONS WITH LAT AND LONG WITH PG'S
import folium
from folium.plugins import MarkerCluster

map_center = [details[0]["latitude"], details[0]["longitude"]]
map_ = folium.Map(location=map_center, zoom_start=8)
marker_cluster = MarkerCluster().add_to(map_)

for doc in details:
      group_name = doc.get("group_name") or doc.get("group_name/name", "Unknown Group")
      folium.Marker(
          location=[doc["latitude"], doc["longitude"]],
          popup=f"""
          <b>Physician:</b> {doc['physician']}<br>
          <b>Group:</b> {group_name}<br>
          <b>Location:</b> {doc['location_name']}<br>
          """,
          tooltip=doc["physician"]
      ).add_to(marker_cluster)

map_.save("PG_GROUP.html")
map_
